In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
HEAD = "https://d37ci6vzurychx.cloudfront.net/trip-data/"

URL_jan = HEAD + "yellow_tripdata_2023-01.parquet"

df_jan = pd.read_parquet(URL_jan)

URL_feb = HEAD + "yellow_tripdata_2023-02.parquet"

df_feb = pd.read_parquet(URL_feb)

print(f"January: Rows: {df_jan.shape[0]}, Columns: {df_jan.shape[1]}")
print(f"Feburary: Rows: {df_feb.shape[0]}, Columns: {df_feb.shape[1]}")

January: Rows: 3066766, Columns: 19
Feburary: Rows: 2913955, Columns: 19


In [ ]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [ ]:
duration = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime

df_jan["duration"] = duration.apply(lambda a: a.total_seconds() / 60)

df_jan["duration"].std()

42.594351241920904

In [ ]:
df_no_outliers = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]

# compute the percentage of records left

df_no_outliers.shape[0]/df_jan.shape[0] * 100

98.1220282212598

In [ ]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

ids = df_no_outliers[['PULocationID', 'DOLocationID']]

one_hot_encoder.fit(ids)

X_train = one_hot_encoder.transform(ids)

X_train.shape

(3009173, 515)

In [ ]:
y_train = df_no_outliers[['duration']]

y_train.shape

(3009173, 1)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

y_pred.shape

(3009173, 1)

In [ ]:
np.sqrt(mean_squared_error(y_train, y_pred))

np.float64(7.649261948684693)

In [ ]:
df_feb.tpep_dropoff_datetime.dtype

dtype('<M8[us]')

In [ ]:
duration = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime

df_feb['duration'] = duration.apply(lambda a: a.total_seconds() / 60)

df2_no_outliers = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [ ]:
X_test = one_hot_encoder.transform(df2_no_outliers[['PULocationID', 'DOLocationID']])

X_test.shape

(2855951, 515)

In [ ]:
y_test = df2_no_outliers[['duration']]

y_test_pred = lr.predict(X_test)

np.sqrt(mean_squared_error(y_test, y_test_pred))

np.float64(7.8118169982078385)